# GAN example

In [1]:
# import necessary packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import random
import sklearn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import LeakyReLU, Reshape, Conv2DTranspose, UpSampling2D, Embedding, Concatenate


In [ ]:
noise_dim = 100 

In [ ]:
import cv2
import os
import numpy as np
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        print(the_class)
        for file in os.listdir(indir+'/'+class_dir):
            #print(file)
            #print("{}/{}/{}".format(indir,class_dir,file))
            if file.endswith('ppm'):
                image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
                image = cv2.resize(image, (64,64))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #image = np.expand_dims(image,axis=2)
                samples.append(image)
                labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels

In [ ]:
#from helper import load_img
samples, labels = load_img('signs4')
print('loaded',len(samples),' samples')
print('classes',set(labels))
samples = samples/255
print(samples.shape)
encoder = sklearn.preprocessing.LabelEncoder() # encoder
labels = encoder.fit_transform(labels)
print(labels[:10])
#print(intlabels[:10])

In [ ]:
import random
plt.figure(figsize=(6, 6))
for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    r = random.randint(0,samples.shape[0])
    plt.imshow(samples[r][:,:,:])
    plt.title(labels[r])
    plt.axis("off")

In [ ]:
# Generator gets a noise vector of size noise_dim and generates an image of size (32 x 32 x 1)
# Our aim: we want the image to be as similar to real images (generated above) as possible
def make_generator_model():
    
    
    input_layer = tf.keras.layers.Input(shape=(noise_dim,))
    x = Dense(8*8*64, use_bias=False)(input_layer)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Reshape((8, 8, 64))(x)

    
    # tu wpiąć generację z klasy = input, embedding(50), reshape 8x8x1 i na koniec concatenate
    input_label = tf.keras.layers.Input(shape=(1,))
    # embedding for categorical input
    y = Embedding(4, 50)(input_label)
    # scale up to image dimensions with linear activation
    y = Dense(8*8)(y)
    y = Reshape((8, 8, 1))(y)
    
    merged = Concatenate()([x, y])
 #   merged = y
    
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', use_bias=False)(merged)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)
    # output: 8 x 8 x 128

    x = UpSampling2D(size=(2, 2), data_format=None, interpolation="nearest")(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)
                    
    # output: 16 x 16 x 64

    x = UpSampling2D(size=(2, 2), data_format=None, interpolation="nearest")(x)
    x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    # output: 32 x 32 x 32

    x = UpSampling2D(size=(2, 2), data_format=None, interpolation="nearest")(x)
    output_layer = Conv2D(3, (3, 3), strides=(1, 1), padding='same', use_bias=False, activation='sigmoid')(x)
    # output: 64 x 64 x 3
    model = tf.keras.models.Model(inputs=[input_layer,input_label], outputs=output_layer)
    model.summary()
    return model



generator = make_generator_model()

In [ ]:
# Discriminator gets image of size (32 x 32 x 1) and decides if it is real or fake
# The result of the discriminator is used by generator to improve 'faking'

def make_discriminator_model():
    
    
    # zacząć od generację z klasy = input, embedding(50), reshape 32x32x1 i na koniec concatenate
    input_label = tf.keras.layers.Input(shape=(1,))
    # embedding for categorical input
    y = Embedding(4, 50)(input_label)
    # scale up to image dimensions with linear activation
    y = Dense(64*64*1)(y)
    y = Reshape((64, 64, 1))(y)
 
    
    
    input_layer = tf.keras.layers.Input(shape=(64, 64, 3))
    
    merged = Concatenate()([input_layer, y])
    x = Conv2D(64, (8, 8), strides=(2, 2), padding='same', input_shape=[64, 64, 4])(merged)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, (8, 8), strides=(2, 2), padding='same')(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    output_layer = Dense(1,activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=[input_layer,input_label], outputs=output_layer)
    #model.add(Dense(1))
    # output: one number 0-fake, 1-real
    model.summary()

    return model

discriminator = make_discriminator_model()

In [ ]:
# returns randomly choosen n samples

def sample_from_dataset(n,samples):
    prev_numbers = []
    new_samples = []
    new_labels = []
    while len(new_samples)<n:
        number = random.randrange(len(samples))
        if number in prev_numbers: continue
        prev_numbers.append(number)
        new_samples.append(samples[number])
        new_labels.append(labels[number])
    new_samples = np.array(new_samples,dtype=float)    
    new_labels = np.array(new_labels)    
    return new_samples,new_labels

In [ ]:
# training parameters
generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)
#cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
cross_entropy = tf.keras.losses.BinaryCrossentropy()


In [ ]:
from numpy.random import randn

def calc_ok(vct):
    ok = 0
    for x in vct: 
        if x>=0.5: 
            ok+=1 
    return ok

# The training step

history = []
##@tf.function
def do_step(images, labels):
    batch_size = len(images)
    images = np.array(images)
    labels = np.expand_dims(labels,axis=1)
    # create random noise for generator
    input_noise = randn(batch_size * noise_dim)
    input_noise = input_noise.reshape(batch_size, noise_dim)
    input_noise = tf.convert_to_tensor(input_noise)
    #noise = tf.random.normal([batch_size, noise_dim])

    random_labels = np.random.randint(4, size=batch_size)
    random_labels = np.expand_dims(random_labels,axis=1)
  #  print(input_noise.shape)
  #  print(random_labels.shape)
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # generate fake image using noise
        generated_images = generator([input_noise,random_labels], training=True)
        # evaluate fake images
        fake_output = discriminator([generated_images,random_labels], training=True)
        fake_acc = (batch_size-calc_ok(fake_output))/batch_size
        # generator want all images to be accepted (output=1)!
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        
        # evaluate real images
        real_output = discriminator([images,labels], training=True)
        real_acc = calc_ok(real_output)/batch_size
        # discriminator wants to classify all real images as 1 and fake images as 0
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        disc_loss = (real_loss + fake_loss)/2 # sum up both losses

    # calculate how to change generator to minimze its loss
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables) # step 4. calculate the gradient of the losses
    # calculate how to change discriminator to minimze its loss
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # update weights for both networks
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables)) # step 5. Apply the optimizers and update weights
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    if epoch % 10 == 0:
        print("Epoch",epoch,'g_loss=',gen_loss.numpy(),'d_loss=',disc_loss.numpy(),"real_acc=",real_acc,"fake_acc=",fake_acc)
    history.append([gen_loss.numpy(),disc_loss.numpy(),real_acc,fake_acc])
    

In [ ]:
epochs = 40000
for epoch in range(epochs):
    # take some random samples
    new_samples,new_labels = sample_from_dataset(50,samples)
    #print(new_samples.shape)
    # perform one training step (epoch)
    do_step(new_samples, new_labels)
    
    if epoch % 100 == 0:
        # show one real image and some fake images generated by generator using noise seed
        #display.clear_output(wait=True)
        num_examples_to_generate = 6
        seed = tf.random.normal([num_examples_to_generate, noise_dim])
        random_labels = np.random.randint(4, size=num_examples_to_generate)
        random_labels = np.expand_dims(random_labels,axis=1)
        predictions = generator([seed,random_labels], training=False)
        fig = plt.figure(figsize=(10,10))
        r = random.randrange(len(samples))
        plt.subplot(1, num_examples_to_generate+1, 1)
        plt.imshow(samples[r, :, :, :])
        plt.title(labels[r])
        plt.axis('off')
        for i in range(predictions.shape[0]):
            plt.subplot(1, num_examples_to_generate+1, i+2)
            #plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.imshow(predictions[i, :, :, :])
            plt.title("{}".format(random_labels[i]))
            plt.axis('off')
        plt.show()    
print("Done")

In [ ]:
nph = np.array(history)

plt.plot(nph[:,0], label='g-loss')
plt.plot(nph[:,1], label='d-loss')
#plt.ylim([0,2])
plt.legend()
plt.show()

In [ ]:
nph = np.array(history)
plt.figure(figsize=(15, 5))
ax = plt.subplot(1, 2, 1)
#plt.ylim([0,2])
plt.plot(nph[:,0], label='generator-loss')
plt.legend()
ax = plt.subplot(1, 2, 2)
plt.plot(nph[:,1], label='discriminator-loss')
#plt.ylim([0,2])
plt.legend()
plt.show()

In [ ]:
nph = np.array(history)

plt.plot(nph[:,2], label='acc-real')
plt.plot(nph[:,3], label='acc-fake')
plt.legend()
plt.show()

In [ ]:
#generator.save_weights('generator_color_4c_40k_weights.h5')

In [ ]:
#generator.save('generator_color_4c_40k')

In [ ]:
#discriminator.save_weights('discriminator_4c_80k_weights.h5')

In [ ]:
#discriminator.save('discriminator_4c_40k.h5')

In [ ]:
noise_dim = 100
num_examples_to_generate = 36
seed = tf.random.normal([num_examples_to_generate, noise_dim])
random_labels = np.random.randint(4, size=num_examples_to_generate)
random_labels = np.expand_dims(random_labels,axis=1)

print(seed.shape)
print(random_labels.shape)
predictions = generator([seed,random_labels])
fig = plt.figure(figsize=(10,10))
# r = random.randrange(len(samples))
# plt.subplot(1, num_examples_to_generate+1, 1)
#plt.imshow(samples[r, :, :, 0] * 127.5 + 127.5, cmap='gray')
#plt.axis('off')
for i in range(predictions.shape[0]):
    plt.subplot(6, 6, i+1)
    #plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    plt.imshow(predictions[i, :, :, :])
    plt.title("{}".format(random_labels[i]))
    plt.axis('off')
plt.show()    